## Einrichtung von Julia JuMP
---

Überprüfen Sie die bereits installierten Pakete. Es sind möglicherweise alle notwendigen Pakete bereits vorhanden.

In [1]:
using Pkg;
Pkg.status()

    Status `C:\Users\Kristian\.julia\environments\v1.2\Project.toml`
  [60bf3e95] GLPK v0.11.4
  [7073ff75] IJulia v1.20.0
  [4076af6c] JuMP v0.20.0
  [79b45036] Remark v0.1.0
  [f3b207a7] StatsPlots v0.12.0


Installieren Sie ``JuMP`` , mit dem Sie einfach Optimierungsprogramme formulieren können. Sowie ``GLPK`` einen open-source Solver zum Lösen der Problem und ``StatsPlots`` zur Visualisierung der Lösung.



In [2]:
#Pkg.add("JuMP")
#Pkg.add("GLPK");
#Pkg.add("StatsPlots");

Initialisieren Sie die installierten Pakete, damit sie im folgenden Notebook verwendet werden können.


In [3]:
using JuMP, GLPK, StatsPlots;

# Aggregierte Planung

### Entscheidungsvariablen:

Bestimmung der optimalen **Produktionsmengen, Absatzmengen, Fremdbezugsmengen** und **Lagerbestände** der Produkte, sowie der optimal zu nutzenden **Zusatzkapazität** der Ressourcen je Periode, 

### Zielfunktion:

0) **Deckungsbeitragsmaximierung**: $$  \qquad \max db = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (e_iA_{it}-k_i^vX_{it}-k_i^l L_{it}-k_i^fF_{it}) - \sum_{j=1}^J \sum_{t=1}^T (k_j^zZ_{jt}) $$


### Nebenbedingungen:

1) **Kapazitaetsrestriktion:** $ \hspace{41mm} \sum_{i=1}^I(r_{ij}\cdot X_{it}) \leq c_j + z_{jt} \hspace{40mm} \forall j \in J, t \in T $

2) **Absatzobergrenze:** $\hspace{38mm} \qquad A_{it} \leq d_{it} \hspace{69mm} \forall i \in I , t \in T $


3) **Absatzuntergrenze:** $ \hspace{37mm}\qquad A_{it} \geq d_{it}^{min} \hspace{66mm} \forall i \in I , t \in T $
 
4) **Lagerbilanz:** $ \hspace{61mm} L_{it}=L_{i,t-1}+X_{it}-A_{it}+F_{it} \hspace{31mm} \forall i \in I , t \in T$

5) **Anfangslagerbestand:**   $ \hspace{43mm} L_{i,0} = l_{start} \hspace{64mm} \forall  i \in I$


6) **Max. Zusatzkapazität:** $ \hspace{44mm} Z_{jt} \leq z_j^{max}  \hspace{65mm} \forall j \in J, t \in T $

7) **Max. Fremdbezug:** $ \hspace{49mm} F_{it} \leq f_i^{max} \hspace{65mm} \forall i \in I, t \in T $

8) **Nichtnegativitätsbedingung:** $ \hspace{31mm} X_{it}, A_{it}, L_{it}, F_{it} \geq 0 \hspace{49mm} \forall i \in I, t \in T$

und $\hspace{81mm} Z_{jt} \geq 0 \hspace{71mm}  \forall j \in J, t \in T $ 

* * *

## Verwendete Symbole

### Mengen

$i \in (1,..,I)\hspace{20mm} $ Produkte  

$j \in (1,..,J)\hspace{20mm}$  Ressourcen 

$t \in (1,..,T)\hspace{20mm}$  Zeiten 

### Variablen

$X_{it}$  $\geq0$      $\hspace{28mm}$ Produktionsmenge      

$A_{it}$  $ \geq 0 $  $\hspace{28mm} $ Absatzmenge 

$L_{it}$  $\geq0$  $\hspace{28mm} $ Lagerbestand  

$F_{it}$       $ \geq0$    $\hspace{28mm} $ Fremdbezug   

$Z_{jt}$      $\geq0$      $\hspace{27mm} $ Zusatzkapazität  


### Parameter

$e_i \hspace{39mm}$   Erlös    

$k_i^v \hspace{38mm}$ variable Kosten

$k_i^l \hspace{38mm}$ Lagerkostensatz 

$k_j^z \hspace{38mm}$ Überstundenkostensatz 

$k_i^f \hspace{38mm}$ Fremdbezugskosten 

$d_{it} \hspace{38mm}$  Nachfrage  

$d_{it}^{min} \hspace{36mm}$ Absatzuntergrenze  

$r_{ij} \hspace{38mm}$  Produktionskoeffizient 

$c_j \hspace{39mm}$  Kapazität  

$l_{start} \hspace{34mm}$  Anfangslagerbestand  

$z_j^{max} \hspace{34mm}$ max. Zusatzkapazität 

$f_i^{max} \hspace{33mm}$  max. Fremdbezug

 * * *
 

## Erstellen Sie das Modell namens ``m`` und geben Sie als zu verwendenden Solver GLPK an.
---

In [4]:
m = Model(with_optimizer(GLPK.Optimizer));

### Mengen und Parameter
---

Fügen Sie die Mengen ein.

In [24]:
#Mengen
Produkte = []  
Ressourcen = []
#In unserem Fall gibt es nur eine Ressource
Perioden = []

#Längen
I = length(Produkte);
J = length(Ressourcen);
T = length(Perioden);

Fügen Sie die Parameter ein.

In [25]:
r = [];
#Ressourcenverbrauch von Ressource j durch Produkt i
#Hinweis: Einheiten beachten (Stunden/Minuten)

       
d = [];
#Nachfrage von Produkt i in Periode t
 
        
dmin = d
#Absatzuntergrenze von Produkt i in Periode t
#Hinweis: Die Nachfrage muss komplett erfüllt werden

0-element Array{Any,1}

In [26]:
c = [];                   #Kapazität der Ressource j
#Hinweis: Gehen Sie vereinfachend davon aus, dass ein Monat 4 Wochen hat.

e = [];                   #Erlös des Produktes

kf =   [];                #Fremdbezugskostensatz der Produkte 
#Hinweis: hier nur langfr. Fremdbezug: günstigeren Kostensatz verwenden

kl =  [];                 #Lagerkostensatz der Produkte

kz =   [];                #Überstundenkostensatz von Ressource j

kv =  [];                 #variable Herstellkosten der Produkte

zmax =  [];               #Max. Zusatzkapazität von Ressource j

l_start =   [];           #Anfangslagerbestand der Produkte


           
fmax = [];  
                          #Maximaler Fremdbezug von Produkt i in Periode t

### Entscheidungsvariablen
---
Definieren Sie die Entscheidungsvariablen. Achten Sie auf die Definitionsbereiche:

 **Nichtnegativitätsbedingung**: Die Entscheidungsvariablen, also die Produktions- und Absatzmengen, Lagerbestände, Fremdbezugsmengen und die in Anspruch genommenen Zusatzkapazitäten dürfen nur positive Werte annehmen, wir setzen deren Nichtnegativität voraus.

$ \qquad X_{it}, A_{it}, L_{it}, F_{it} \geq 0 \qquad \qquad \forall i \in I, t \in T$

und $\qquad \qquad Z_{jt} \geq 0 \qquad \qquad \qquad  \forall j \in J, t \in T $ 

In [27]:
@variables  m begin
            X[1:I,1:T]>=0,Int  #Produktionsmenge von Produkt i in Periode t
            A[1:I,1:T]>=0,Int  #Absatzmenge von Produkt i in Periode t
            F[1:I,1:T]>=0,Int  #Fremdbezugsmenge von Produkt i in Periode t
            Z[1:J,1:T]>=0,Int  #Zusatzkapazität von Ressource j in Periode t
            L[1:I,0:T]>=0,Int; #Lagerbestand von Produkt i am Periodenende von t
            end

## Zielfunktion

 **Deckungsbeitragsmaximierung:** Der Gesamtdeckungsbeitrag db soll maximiert werden. Dieser berechnet sich hier aus der Summe der einzelnen Produkterlöse abzüglich der variablen Kosten, abzüglich der Lagerkosten, abzüglich der Fremdbezugskosten und wird Zusätzlich noch verringert um die Zusatzkapazitätskosten der Ressourcen.

$  \qquad \max db = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (e_iA_{it}-k_i^vX_{it}-k_i^l L_{it}-k_i^fF_{it}) - \sum_{j=1}^J \sum_{t=1}^T (k_j^zZ_{jt}) $

In [28]:
@objective(m, Max,  sum(e[i] * A[i,t] - kv[i] * X[i,t] - kl[i] * L[i,t] - kf[i] * F[i,t] 
           for i=1:I for t=1:T) - sum(kz[j] *Z[j,t] for j=1:J for t=1:T));

ErrorException: The objective function `Val{false}()` is not supported by JuMP.

## Nebenbedingungen

**Kapazitätsrestriktion**: Die zur Verfügung stehende Kapazität für die auf den einzelnen Ressourcen mögliche Produktionszeit darf nicht überschritten werden. Die Summe der Gesamtproduktionszeit aller Produkte $ i = 1,...,I $ auf den einzelnen Ressourcen j darf in jeder Periode die jeweils verfügbare Kapazität  $𝑐_j$ ergänzt um die Zusatzkapazität $o_{jt}$  nicht überschreiten.

$ \qquad \sum_{i=1}^I(r_{ij}\cdot X_{it}) \leq c_j + z_{jt} \qquad \forall j \in J, t \in T $

In [29]:
@constraint(m, KapRes[t=1:T,j=1:J], sum(r[i,j] * X[i,t] for i=1:I) <= c[j] + Z[j,t]);

 **Absatzuntergrenze**: Für die Produkte existieren Absatzuntergrenzen basierend auf der Nachfrage. Die abgesetzte Menge der einzelnen Produkte kann diese Absatzuntergrenze in keiner Periode unterschreiten.

$ \qquad A_{it} \geq d_{it}^{min} \qquad \qquad \qquad \qquad \forall i \in I , t \in T $

In [30]:
@constraint(m, AbsUn[i=1:I,t=1:T], A[i,t] >= dmin[i,t]);

 **Absatzobergrenze**: Für die Produkte existieren Absatzober- und untergrenzen basierend auf der Nachfrage. Die abgesetzte Menge der einzelnen Produkte kann diese Absatzobergrenze in keiner Periode überschreiten.

$ \qquad A_{it} \leq d_{it}^{max} \qquad \qquad \qquad \qquad \forall i \in I , t \in T $

In [31]:
@constraint(m, AbsOb[i=1:I,t=1:T], A[i,t] <= d[i,t]);

 **Lagerbilanz**: Der Lagerbestand eines Produktes am Ende einer Periode berechnet sich aus der Summe der eingelagerten Menge in der Vorperiode, der Produktionsmenge und der Fremdbezugsmenge, abzüglich der abgesetzen Menge des Produktes.

$ \qquad L_{it}=L_{i,t-1}+X_{it}-A_{it}+F_{it} \qquad \forall i \in I , t \in T$

In [32]:
#Füge den fehlenden Code ein

**Anfangslagerbestand**: Der Anfangslagerbestand aller Produkte entspricht dem initial gesetzen $l_i$.
 
  $$ \qquad L_{i,0} = l_{start} \hspace{40mm} \forall  i \in I$$


In [33]:
@constraint(m, AnfLager[i=1:I], L[i,0] == l_start[i]);

 **Max. Zusatzkapazität**: Die eingesetzten Zusatzkapazitäten (Überstunden) für jede Ressource müssen in jeder Periode unter der Obergrenze, der maximalen Zusatzkapazität, bleiben. Die maximale Zusatzkapazität ist für jede Ressource j einzeln gegeben.

$ \qquad Z_{jt} \leq z_j^{max}  \qquad \qquad \qquad \qquad \forall j \in J, t \in T $

In [34]:
@constraint(m, ZusatzKapa[j=1:J,t=1:T], Z[j,t] <= zmax[j] );

 **Max. Fremdbezug**: Der dazugekaufte Fremdbezug jedes Produktes muss in allen Perioden unter der Obergrenze, dem maximalen Fremdbezug, bleiben. Der maximale Fremdbezug ist für jedes Produkt i einzeln gegeben. 

$ \qquad F_{it} \leq f_i^{max} \qquad \qquad \qquad \qquad \forall i \in I, t \in T $

In [35]:
@constraint(m, MaxFB[i=1:I,t=1:T], F[i,t] <= fmax[i,t] );

## Lösen Sie das Modell.
---

In [36]:
#Befehl zum Lösen des Modells einfügen

Lassen Sie sich das Modell anzeigen.

In [37]:
#Befehl zum Anzeigen des Modells einfügen

Lassen Sie sich den Zielfunktionswert anzeigen.

In [38]:
db = #Befehl zum Anzeigen des Zielfunktionswertes einfügen

println("Objective value db: ", round(Int64,db))

Beispiel zur Berechnung der Erlöse der abgesetzten Herren-Sonnenbrillen:

In [1]:
round(Int64,sum(JuMP.value.(A[2,t])*e[2] for t in 1:T))

UndefVarError: UndefVarError: T not defined

Platz für weitere Berechnungen: